# Milestone 4 Checkpoint

## StocksFlags
StocksFlags is a multi-agent AI equity analysis tool designed to provide comprehensive insights into the stock market. It leverages Large Language Models (LLMs) and agentic workflows to automate data gathering, processing, and analysis, ultimately generating investment signals and supporting user queries through a chatbot interface.

### Key Technologies

*   **Programming Language:** Python
*   **AI/LLM Frameworks:** LangChain, LangGraph
*   **Data Sources:** Multiple APIs (e.g., Polygon, Yahoo Finance), Custom MCP Integration, WRDS (Wharton Research Data Services)
*   **Database:** MySQL

### Core Functionality

The project aims to deliver functionality across three core components:

1.  **LLM Agent-Driven Analysis Tool:**
    *   Acts as the primary user interface for research and analysis.
    *   Accepts user queries in natural language.
    *   Utilizes LLM agents to autonomously:
        *   Determine the required information.
        *   Select and execute appropriate data retrieval tools (fetching market data, news, fundamentals from APIs like Polygon, Yahoo Finance, WRDS, and the internal database).
        *   Process the retrieved information using defined analysis tools (e.g., summarization, sentiment analysis, event extraction).
        *   Synthesize findings to answer the user's query.
    *   Integrates with the other components (Trading Signals and Valuation Model) as callable tools within its workflow.

2.  **LLM Agent-Driven Trading Signal Generation:**
    *   Employs specialized LLM agents (`src/agent`) to generate investment/trading signals.
    *   Agents analyze data based on various strategies:
        *   Simulating renowned investor approaches.
        *   Fundamental analysis.
        *   Macroeconomic assessment.
        *   Technical analysis.
        *   News sentiment interpretation.
    *   Can be executed directly or invoked as a tool by the main Analysis Tool.

3.  **Damodaran Valuation Model:**
    *   Implements valuation methodologies inspired by Professor Aswath Damodaran.
    *   Provides a user interface (details TBD) for manual input and valuation generation.
    *   Offers a callable tool interface, allowing the LLM Agent-Driven Analysis Tool to programmatically generate valuations as part of its research process.



## Important Notes for Graders
- **DO NOT RUN THIS NOTEBOOK**: The source code are not accessible when submitting a single notebook
- This is a demonstration of a part of our langraph workflow
- The complete codebase can be accessed at [GitHub Repository](https://github.com/Chen-zexi/StocksFlags)

We welcome any feedback on our data selection approach and integration methodology.

### Langrah
**A trace of this run can be accessed through** [langsmith](https://smith.langchain.com/public/ceac802f-97b9-4630-855e-3e8a34befb55/r)

Below is an image demonstrate the current agent workflow
![graph](../../assets/graph.jpg)

In [15]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from langchain_google_genai import ChatGoogleGenerativeAI
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage
from langgraph_sdk import get_client

import logging
import uuid


In [16]:
client = get_client(url="http://localhost:2024")

In [17]:

user_query = ["What are some most recent news of Nvidia?"]

user_input_messages = user_query



MAX_CACHE_SIZE = 2

streaming_llm_agents = [*TEAM_MEMBERS, "planner", "coordinator"]

# Reset coordinator cache at the start of each workflow
global coordinator_cache
coordinator_cache = []
global is_handoff_case
is_handoff_case = False

thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)



HumanMessage(
    content='What are some most recent news of Nvidia?',
    additional_kwargs={},
    response_metadata={},
    id='636cdbca-e2ce-4c29-a9ea-89d57df14133',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants to know the most recent news about Nvidia. I need to gather the latest news arti'
        'cles and updates related to Nvidia, summarize the findings, and present them in a professional report.",\n'
        '  "title": "Gather and Report the Most Recent News about Nvidia",\n'
        '  "steps": [\n'
        '    {\n'
        '      "agent_name": "researcher",\n'
        '      "title": "Collect the Most Recent News about Nvidia",\n'
        '      "description": "Use search engines and news retrieval tools to gather the latest news articles and upda'
        'tes about Nvidia. Summarize the key points and developments from the most recent sources."\n'
        '    },\n'
        '    {\n'
        '      "agent_name": "reporter",\n'
      

In [18]:
current_state = await client.threads.get_state(thread["thread_id"])

## Final state of workflow

The output below demonstrate the Final State of the workflow, including the shared memory across agents. The final response is stored in the last messsage generated by **reporter agent**

In [24]:
current_state['values']['messages']

[{'content': 'What are some most recent news of Nvidia?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': '636cdbca-e2ce-4c29-a9ea-89d57df14133',
  'example': False},
 {'content': '{\n  "thought": "The user wants to know the most recent news about Nvidia. I need to gather the latest news articles and updates related to Nvidia, summarize the findings, and present them in a professional report.",\n  "title": "Gather and Report the Most Recent News about Nvidia",\n  "steps": [\n    {\n      "agent_name": "researcher",\n      "title": "Collect the Most Recent News about Nvidia",\n      "description": "Use search engines and news retrieval tools to gather the latest news articles and updates about Nvidia. Summarize the key points and developments from the most recent sources."\n    },\n    {\n      "agent_name": "reporter",\n      "title": "Present a Professional Report on Recent Nvidia News",\n      "description": "Write a professional repor

## Formate the response with Markdown

In [23]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['messages'][-1]['content']))

# Recent News Report: Nvidia (April 2025)

## Executive Summary

This report provides an overview of the most recent news concerning Nvidia, focusing on product launches, financial performance, regulatory challenges, and executive actions. The information is based on news articles published in April 2025 from reputable sources.

---

## Key Findings

- **Product Launches**: Nvidia, in partnership with Gigabyte, has introduced the GeForce RTX 5060 series, featuring the new Blackwell architecture and DLSS 4.
- **AI Infrastructure Expansion**: Nvidia’s DGX Blackwell B200 cluster is powering new GPU-as-a-Service offerings in the EMEA region.
- **Financial Impact**: Nvidia’s stock has declined over 30% from its January peak, primarily due to US tariffs and export restrictions affecting AI chip sales to China.
- **Executive Engagement**: CEO Jensen Huang has visited Beijing for high-level talks following new US export controls.
- **Export Ban Consequences**: Nvidia reported a $5.5 billion charge this quarter due to the US AI chip export ban to China, resulting in a 6.9% drop in stock price.

---

## Detailed Analysis

### 1. Product Innovation and Launches

- **Gigabyte GeForce RTX 5060 Series**: Gigabyte has launched the GeForce RTX 5060 series GPUs, utilizing Nvidia’s Blackwell RTX architecture and supporting DLSS 4. These GPUs are aimed at the mid-range gaming market, specifically for 2K and 1080p gaming.
  - *Source: gadgets360.com*

### 2. AI Ecosystem and Enterprise Expansion

- **PEAK:AIO and Nvidia DGX B200**: PEAK:AIO is supporting Scan Computers’ expansion of GPU-as-a-Service in the EMEA region using Nvidia’s DGX Blackwell B200 cluster. This move highlights Nvidia’s growing presence in AI infrastructure and enterprise solutions.
  - *Source: storagenewsletter.com*

### 3. Financial and Regulatory Challenges

- **Stock Performance and Tariffs**: Nvidia’s stock has dropped over 30% from its January 2025 peak, largely due to concerns over US tariffs and export restrictions, particularly those affecting AI chip sales to China. The company faces a $5.5 billion financial hit this quarter as a result.
  - *Source: wealthcreationinvesting.com, watcher.guru*

- **Export Ban Impact**: The US government’s new controls on AI chip exports to China have led to a significant financial charge for Nvidia and increased uncertainty regarding future revenue from the Chinese market.
  - *Source: watcher.guru*

### 4. Executive Actions

- **CEO Visit to China**: Nvidia CEO Jensen Huang has traveled to Beijing for discussions with Chinese officials following the implementation of new US export restrictions. This demonstrates Nvidia’s proactive approach to managing geopolitical and regulatory risks.
  - *Source: irishtimes.com*

---

## Conclusions and Recommendations

### Conclusions

- Nvidia is facing significant short-term challenges due to US export bans and tariffs, which have negatively impacted its stock price and financial outlook.
- Despite these headwinds, Nvidia continues to innovate and expand its product and AI infrastructure offerings.
- The company’s leadership is actively engaging with stakeholders in China to address regulatory challenges.

### Recommendations

- **Monitor Regulatory Developments**: Stakeholders should closely follow updates on US-China trade relations and export controls, as these will continue to impact Nvidia’s business.
- **Evaluate Long-Term Prospects**: While short-term sentiment is negative, Nvidia’s ongoing innovation and enterprise expansion may support long-term growth.
- **Assess Market Volatility**: Investors should be aware of continued volatility in Nvidia’s stock due to geopolitical and regulatory uncertainties.

---

**Sources**: gadgets360.com, storagenewsletter.com, wealthcreationinvesting.com, irishtimes.com, watcher.guru

*Information is current as of April 17, 2025. No additional data outside the provided sources has been used.*
